In [6]:
import pyart
from matplotlib import pyplot as plt
import numpy as np
from numpy import dtype
import os
from netCDF4 import Dataset,date2num,num2date
import metpy.calc
from metpy.units import units
import warnings
warnings.filterwarnings('ignore')
from wradlib.trafo import r_to_depth
import datetime as dt
from scipy.stats import mstats
import numpy.ma as ma

In [7]:
class UF:
    def __init__(self,radar,shape,lat_0, lon_0):
        self.radar = radar
        self.shape = shape
        self.lat_0 = lat_0
        self.lon_0 = lon_0
    def calculate_attenuation_zphi(self):
        fixed_fzl=3000
        spec_at, pia_dict, cor_z, spec_diff_at, pida_dict, cor_zdr= pyart.correct.calculate_attenuation_zphi(
            self.radar,
            refl_field='reflectivity', 
            phidp_field= 'differential_phase',
            zdr_field='differential_reflectivity',
            temp_ref='fixed_fzl')
        return cor_z
    def noise_rejection(self):
        range_bins=self.radar.range['data']
        ref_last_bins=self.radar.fields['reflectivity']['data'][:,-5:]
        Z_f=mstats.mode(ref_last_bins,axis=None)[0][0]
        Z_s=pyart.retrieve.compute_noisedBZ(360, noisedBZ_val=Z_f, _range=range_bins, ref_dist=300., noise_field='noise_dbz')
        Z_splus4=Z_s['data']+4
        conditions_masked=ma.masked_less(self.radar.fields['reflectivity']['data'],Z_splus4)
        self.radar.add_field_like('reflectivity', 'reflectivity_cor_noise',ma.masked_less_equal(conditions_masked,0.),replace_existing=True)
    def convert_grid(self):
        grid = pyart.map.grid_from_radars(
            self.radar,
            grid_shape=self.shape, #Number of points in the grid (z, y, x)
            grid_limits=((0, 12000), (-300000, 300000), (-300000, 300000)), # min-max tuong duong z,y,x
            grid_origin = (self.lat_0, self.lon_0),
            fields=['rain_R_Z'])
        return grid
    def convert_lat_lon(self, grid):
        for i in range(self.shape[1]):
            geog = pyart.core.cartesian_to_geographic_aeqd(grid.x["data"][i],grid.y["data"][i],self.lon_0, self.lat_0, R=6370997.0)
            grid.x["data"][i] = geog[0]
            grid.y["data"][i] = geog[1]
        return grid
    def create_nc(self, fileout, grid, time_unit, time_calendar):
        file = Dataset(fileout,'w')
        file.title = "Netcdf radar file - Made by PYTHON"
        file.createDimension('longitude',self.shape[2])
        file.createDimension('latitude',self.shape[1])
        file.createDimension('lev',self.shape[0])
        file.createDimension('time',1)
        times = file.createVariable('time',dtype('f4').char,('time',))
        lons = file.createVariable('longitude',dtype('f4').char,('longitude',))
        lats = file.createVariable('latitude',dtype('f4').char,('latitude',))
        lev = file.createVariable('lev',dtype('f4').char,('lev',))
        lats.units = 'degrees_north'
        lats.standard_name = "Latitude"
        lats.long_name = "Latitude"
        lats.axis = "Y"
        lons.standard_name = "Longitude"
        lons.long_name = "Longitude"
        lons.axis = "X"
        lons.units = 'degrees_east'
        lev.units = 'meter'
        lev.axis = "Z"
        times.units = time_unit #'time_unit'
        times.standard_name = "time"
        times.calendar = time_calendar
        times.axis = "T"
        lons[:] = grid.x["data"]
        lats[:] = grid.y["data"]
        lev[:] = grid.z["data"]
        #dates=[dt.datetime(2021,7,2,12,40,5)]
        #time = date2num(dates, times.units)
        times[:] = 0 #time_unit 
        ref = file.createVariable('precipitation',dtype('f4').char,('time','lev','latitude','longitude'))
        ref.units = 'mm'
        ref[:] = grid.fields['rain_R_Z']['data']
        file.close()

In [8]:
linkUF="C:/Users/Admin/Desktop/test_py_WRADLIB/"
linkOUT="C:/Users/Admin/Desktop/test_py_WRADLIB/"

In [9]:
def main(filein, fileout):
    radar = pyart.io.read_sigmet(filein)
    radar=radar.extract_sweeps([0]) # axtract sweep 0,5 
    site_name = radar.metadata['instrument_name'] 
    radar_name = radar.metadata['instrument_name']
    time_unit = radar.time['units'] #[14:30]
    time_calendar = radar.time['calendar']
    range_ = radar.range #300km (Khoang cach tinh tu trung tam)
    scan_type = radar.scan_type  #PPI
    project = radar.projection
    altitude = radar.altitude #36m tren sea level
    azimuth = radar.azimuth['data'] #len=6948 phuong vi cua anten
    elevation = radar.elevation['data'] #len=6948*240 do cao goc
    gate_x = radar.gate_x['data'][0] #meter len=6948 toa do decac
    gate_y = radar.gate_y['data'][0]
    gate_z = radar.gate_z['data'][0] #meter
    latitude = radar.gate_latitude['data']
    longitude = radar.gate_longitude['data']
    lat_0 = radar.latitude['data'][0]
    lon_0 = radar.longitude['data'][0]
    shape = (1, 601, 601)
    UF_File = UF(radar, shape, lat_0, lon_0)
    #cor_z=UF_File.calculate_attenuation_zphi()
    #radar.add_field('corrected_reflectivity_zphi', cor_z)
    #gatefilter = pyart.filters.GateFilter(radar)
    #gatefilter.exclude_below('reflectivity', 10)
    ref_cor_noise=UF_File.noise_rejection()
    rain_R_Z=pyart.retrieve.est_rain_rate_z(radar,alpha=0.0376, beta=0.6112,refl_field='reflectivity')#,gatefilter=gatefilter)
    radar.add_field('rain_R_Z', rain_R_Z)
    grid = UF_File.convert_grid() 
    grid = UF_File.convert_lat_lon(grid) # convert to lat-lon grid
    grid.fields['rain_R_Z']['data']=r_to_depth(grid.fields['rain_R_Z']['data'],600) # convert mm/h to mm interval 10p
    UF_File.create_nc(fileout[0:-2]+ ".nc", grid, time_unit, time_calendar)

In [10]:
for name in os.listdir(linkUF):
    filein = linkUF + name
    fileout = linkOUT + os.path.splitext(name)[0]
    main(filein, fileout)